In [ ]:
import pandas as pd
import MySQLdb

In [ ]:
db = MySQLdb.connect(
'localhost',
'root',
'1234',
charset='utf8')

In [ ]:
cursor = db.cursor()

In [ ]:
sql = '''
show databases;
'''
cursor.execute(sql)

In [ ]:
cursor.fetchall()

In [ ]:
# 데이터베이스 생성 - budongsan

sql = '''
create database budongsan;
'''
cursor.execute(sql)

In [ ]:
sql = '''
show databases;
'''
cursor.execute(sql)

In [ ]:
cursor.fetchall()

In [ ]:
db = MySQLdb.connect(
'localhost',
'root',
'1234',
'budongsan',
charset='utf8')

In [ ]:
# name , addr, height, email

In [ ]:
sql = '''
use budongsan;
'''
cursor.execute(sql)

In [ ]:
sql = '''
create table mytable (
name varchar(20),
addr varchar(20),
height int,
email varchar(20)
)
'''
cursor.execute(sql)

In [ ]:
sql = 'show tables;'

cursor.execute(sql)
cursor.fetchall()

In [ ]:
sql = '''
insert into mytable values('park','seoul',170, 'multi@gamil.com')
'''
cursor.execute(sql)

In [ ]:
sql = '''
select * from mytable;
'''
cursor.execute(sql)
cursor.fetchall()

In [ ]:
df = pd.read_csv('budongsan.csv')
df.head()

In [ ]:
sql = '''
create table room(
addr varchar(225),
agent varchar(225),
deposit int,
rent int
)
'''
cursor.execute(sql)

In [ ]:
df

In [ ]:
for i in range(len(df)):
    addr = df.iloc[i][3]
    agent = df.iloc[i][4]
    deposit = df.iloc[i][5]
    rent = df.iloc[i][6]
    
    sql = '''
    insert into room values('{}', '{}', {}, {})
    '''.format(addr, agent, deposit, rent)
    
    cursor.execute(sql)    
db.commit()

In [ ]:
sql = ' select * from room;'
cursor.execute(sql)
cursor.fetchall()

In [ ]:
# sakila db
# 렌탈 횟수가 30회 이상인 유저
# 유저 이름, 유저 이메일
# sql, pandas
# customer, rental

In [ ]:
db = MySQLdb.connect(
'localhost',
'root',
'1234',
'sakila',
charset='utf8')

In [ ]:
sql = '''
select first_name, email, count(rental_id) num_rent

from customer c join rental r
on c.customer_id = r.customer_id

group by c.customer_id

having num_rent > 30

order by num_rent desc;
'''
pd.read_sql(sql, db).head()

In [ ]:
sql = '''
select * from customer;
'''

customer_df = pd.read_sql(sql,db)
customer_df.head()

In [ ]:
sql = '''
select * from rental;
'''

rental_df = pd.read_sql(sql,db)
rental_df.head()

In [ ]:
df = pd.merge(customer_df, rental_df, on = 'customer_id')
df.head(2)

In [ ]:
df30 = df.groupby('customer_id')['rental_id'].count()>30
df30

In [ ]:
idx30 = [ idx for idx, i in enumerate(df30) if i == True ]
idx30

In [ ]:
df = df[['first_name','email','customer_id']]

In [ ]:
uniquedf = df.drop_duplicates(['customer_id'], keep='last')

In [ ]:
uniquedf.set_index('customer_id', drop=True).iloc[idx30]

In [ ]:
pwd

In [ ]:
# film 테이블에서 2006년이나 2007년에 출시되었으면서
# 'PG' 등급이거나 'G' 등급의 영화제목
# film table

In [ ]:
sql = '''
select title 
from film
where release_year = 2006 or release_year = 2007
and rating = 'pg' or rating = 'g'
;
'''
pd.read_sql(sql, db)

In [ ]:
sql = '''
select * from film;
'''

film_df = pd.read_sql(sql, db)
year67 = (film_df['release_year'] == 2006) | \
           (film_df['release_year'] == 2007)

In [ ]:
pgg = (film_df['rating'] == 'PG') | (film_df['rating'] == "G")

In [ ]:
film_df[year67 & pgg]

In [ ]:
# user별 매출
# customer, payment
# customer_id, 매출 내림차순

In [ ]:
sql = '''
select 
  c.customer_id, 
  sum(p.amount) 'revenue'
from
  payment p
  join customer c
on c.customer_id = p.customer_id
group by customer_id
order by revenue desc;
'''
pd.read_sql(sql, db).head()

In [ ]:
customer_df = pd.read_sql('select * from customer;', db)
payment_df = pd.read_sql('select * from payment;', db)

In [ ]:
df = pd.merge(customer_df, payment_df, on ='customer_id')
pd.DataFrame(df.groupby('customer_id')['amount'].sum()).\
    sort_values('amount', ascending=False)